In [ ]:
import fitz
from haystack.components.builders import PromptBuilder
from openai import OpenAI
from env_params import openrouter_api_key
import json
import random


class Quizzer():

    def __init__(self, openrouter_api_key):
        """
        Function to initialize the Quizzer"""
        
        self.openrouter_api_key = openrouter_api_key
        self.random_level = True
        self.random_number = True
        self.template = """Given the following text, create {{no_questions}} multiple choice quizzes in JSON format.
Each question should have 4 different options, and only one of them should be correct.
The questions should be {{question_level}}.
Each question should increase in difficulty.
Each question should be independent of the others.
Each question should be based on the text provided.
Each question should be clear and concise.
The options should be unambiguous.
Each option should begin with a letter followed by a period and a space (e.g., "a. option").
The question should also briefly mention the general topic of the text so that it can be understood in isolation.
Each question should not give hints to answer the other questions.
Include challenging questions, which require reasoning.

respond with JSON only, no markdown or descriptions.

example JSON format you should absolutely follow:
{"topic": "a sentence explaining the topic of the text",
 "questions":
  [
    {
      "question": "text of the question",
      "options": ["a. 1st option", "b. 2nd option", "c. 3rd option", "d. 4th option"],
      "right_option": "c"  # letter of the right option ("a" for the first, "b" for the second, etc.)
    }, ...
  ]
}


text:
{% for doc in documents %}{{ doc.content|truncate(1000000) }}{% endfor %}
"""

    def edit_random_level(self, random_level):
        """_summary_

        Args:
            random_level (_type_): _description_
        """
        self.random_level = random_level

    def edit_random_number(self, random_number):
        """_summary_

        Args:
            random_number (_type_): _description_
        """
        self.random_number = random_number

    def get_question_level(self):
        """
        Function to get the question level
        """
        got_answer = False
        while not got_answer:
            try:
                self.question_level = input("Enter the question level (easy (e), medium (m), hard (h)): ")
                if self.question_level.lower() in ['easy','medium','hard','e','m','h']:
                    got_answer = True
                else:
                    print("Please enter a valid question level")
            except ValueError:
                print("Please enter a valid question level")
        if self.question_level.lower() == 'easy' or self.question_level.lower() == 'e':
            self.question_level = "easy"
        elif self.question_level.lower() == 'medium' or self.question_level.lower() == 'm':
            self.question_level = "medium"
        else:
            self.question_level = "hard"
        #return question_level
    
    def read_pdf(self, pdf_path):
        """_summary_

        Args:
            pdf_path (_type_): _description_

        Returns:
            _type_: _description_
        """
        pdf_document = fitz.open(pdf_path)
        text = ""
        for page_number in range(len(pdf_document)):
            page = pdf_document.load_page(page_number)
            text += page.get_text()
        return text

    def get_list_of_documents(self, documents_paths):
        """
        Function to get the list of documents

        Args:
            documents_paths (_type_): _description_
        """
        self.document_list = []
        for document in documents_paths:
            self.document_list.append({'content':self.read_pdf(document)})
        #return self.document_list

    def get_number_of_questions(self):
        """
        Function to get the number of questions
        """
        got_answer = False
        while not got_answer:
            try:
                self.no_questions = int(input("Enter the number of questions: "))
                got_answer = True
            except ValueError:
                print("Please enter a number")
        #return self.no_questions

    def get_path_for_documents(self):
        """
        Function to get the path for the documents
        """
        got_answer = False
        while not got_answer:
            try:
                self.documents_paths = input("Enter the path of the documents separated by a space: ").split()
                got_answer = True
            except ValueError:
                print("Please enter a valid path")
        #return documents_paths

    def create_prompt(self):
        """
        Function to create the prompt for the OpenRouter API
        """
        self.get_path_for_documents()
        if not self.random_number:
            self.get_number_of_questions()
        else:
            self.no_questions = random.randint(3,7)
        if not self.random_level:
            self.get_question_level()
        else:
            self.question_level = random.choice(["easy", "medium", "hard"])
        self.get_list_of_documents(self.documents_paths)
        print('Using these documents: ' + str(self.documents_paths))
        builder = PromptBuilder(
        template=self.template,
        required_variables=["documents","no_questions","question_level"]
        )
        self.prompt = builder.run(documents=self.document_list, no_questions=self.no_questions, question_level=self.question_level)

    def call_openrouter(self):
        """
        Function to call the OpenRouter API
        """
        print("Please wait a few seconds for the response (average is 40 seconds)")
        print('')
        client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=self.openrouter_api_key,
        )
        completion = client.chat.completions.create(
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        messages=[
            {
            "role": "user",
            "content": self.prompt['prompt']
            }
        ]
        )
        try:
            self.quiz = completion.choices[0].message.content.replace('```json','').replace('```','').replace('```','')
        except:
            self.quiz = completion.choices[0].message.content

    def answer_questions(self):
        """
        Function to answer the questions in the quiz
        """
        correct_answer = 0
        response = json.loads(self.quiz)
        print('The topic is: ' + response['topic'])
        print('')
        for question in response['questions']:
            applicable_options = ["a", "b", "c", "d"]
            got_answer = False
            while not got_answer:
                print(question['question'])
                for option in question['options']:
                    print(option)
                answer = input("Enter the right option: ").lower()
                if answer in applicable_options:
                    got_answer = True
                else:
                    #clear_output(wait = True)
                    #print('',flush=True)
                    print("Please enter a valid option")
            #clear_output(wait = True)
            #print('',flush=True)
            if answer == question['right_option']:
                print("Correct!\n")
                correct_answer += 1
            else:
                print(f"Wrong! The right option is {question['right_option']}\n")
            print('')

        print('')
        print('')
        print(f"Your score is {correct_answer}/{len(response['questions'])}")

    def run(self):
        """
        Main function to run the quizzer
        """
        self.create_prompt()
        self.call_openrouter()
        self.answer_questions()

In [ ]:
quizzer = Quizzer(openrouter_api_key)
quizzer.edit_random_level(True)
quizzer.edit_random_number(True)
quizzer.run()